<a href="https://colab.research.google.com/github/ftempesta/Data-Science-Online/blob/master/Tutorial_3_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# K-Means

Crearemos los datos de evaluaciones de películas:

Cada observación corresponde a los ratings que un usuario dio a ciertas películas, por lo que cada atributo es el rating de una película en particular:

In [ ]:
ratings = [['john', 5, 5, 2, 1], 
           ['mary', 4, 5, 3, 2],
           ['bob', 4, 4, 4, 3],
           ['lisa', 2, 2, 4, 5],
           ['lee', 1, 2, 3, 4],
           ['harry', 2, 1, 5, 5]]

titles = ['user', 'Jaws', 'Star Wars', 'Exorcist', 'Omen']
movies = pd.DataFrame(ratings, columns=titles)
movies

**¿Cuántos clusters observamos?**

---

Vamos a hacer k-means sobre estos datos. Para ello, quitamos la columna `user`, ya que es sólo un identificador.

También podemos usar el índice del DataFrame para identificar al usuario, en vez de usar una columna específica para ello.

In [ ]:
from sklearn import cluster

data = movies.drop('user', axis=1)
k_means = cluster.KMeans(n_clusters=2, max_iter=50, random_state=1)
k_means.fit(data)

labels = k_means.labels_
pd.DataFrame(labels, index=movies.user, columns=['Cluster ID'])

Podemos inspeccionar los centroides resultantes del clustering:

In [ ]:
centroids = k_means.cluster_centers_
pd.DataFrame(centroids, columns=data.columns)

Y podemos incluir una nueva columna con el ID del cluster que obtuvimos para cada usuario.

In [ ]:
movies['Cluster ID'] = labels
movies

Ahora que tenemos un clustering, podríamos usarlo con datos nuevos, es decir, le podríamos pasar nuevas evaluaciones de películas y el clustering nos diría a cuál cluster corresponden, similar a la clasificación:

In [ ]:
import numpy as np

testData = np.array(
    [[4, 5, 1, 2],
     [3, 2, 4, 4],
     [2, 3, 4, 1],
     [3, 2, 3, 3],
     [5, 4, 1, 4]])

labels = k_means.predict(testData)
labels

In [ ]:
newusers = pd.DataFrame(testData)
newusers['user'] = ['paul', 'kim', 'liz', 'tom', 'bill']
newusers['Cluster ID'] = labels

newusers = newusers[['user', 0, 1, 2, 3, 'Cluster ID']]
newusers.columns = movies.columns

newusers

¿Podría haber más de dos clusters?

Usemos el método del codo para determinar si dos clusters fue una decisión acertada.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

numClusters = [1,2,3,4,5,6]
SSE = []
for k in numClusters:
    k_means = cluster.KMeans(n_clusters=k, n_init=10)
    k_means.fit(data)
    SSE.append(k_means.inertia_)

plt.plot(numClusters, SSE)
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')
plt.show()

Observamos que el "codo" se encuentra claramente marcado en dos clusters.

# Clustering Jerárquico Aglomerativo

Carguemos los datos de animales nuevamente:

In [ ]:
ver = pd.read_csv('http://www.cse.msu.edu/~ptan/dmbook/tutorials/tutorial8/vertebrate.csv')
ver

Vamos a visualizar los dendogramas de distintos "linkages" o criterios para clustering jerárquicos.

Acá la variable `Z` se llama el "linkage". Es la matriz de similitud con la información de los clusters que encontró el algoritmo.

In [ ]:
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt
%matplotlib inline

names = ver['Name']
Y = ver['Class']
X = ver.drop(['Name','Class'], axis=1)
Z = hierarchy.linkage(X.values, 'single')
dn = hierarchy.dendrogram(Z, labels=names.tolist(), orientation='right')

In [ ]:
Z = hierarchy.linkage(X.values, 'complete')
dn = hierarchy.dendrogram(Z,labels=names.tolist(),orientation='right')

In [ ]:
Z = hierarchy.linkage(X.values, 'average')
dn = hierarchy.dendrogram(Z,labels=names.tolist(), orientation='right')

# DBScan

Carguemos los siguientes datos en dos dimensiones y grafiquémoslos:

In [ ]:
cam = pd.read_csv('http://www.cse.msu.edu/~ptan/dmbook/tutorials/tutorial8/chameleon.data', delimiter=' ', names=['x', 'y'])
cam

In [ ]:
cam.plot(x='x', y='y', kind='scatter')

Aplicaremos DBScan con valores eps=15.5 y minPts=5:

Esta implementación asocia los puntos de ruido al cluster ID con valor `-1`.

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=15.5, min_samples=5).fit(cam)

In [ ]:
cam['Cluster ID'] = db.labels_

cam

In [ ]:
cam.plot(x='x', y='y', c='Cluster ID', colormap='jet', kind='scatter')

# Preprocesamiento

In [ ]:
from sklearn import preprocessing 

In [ ]:
cam = cam.drop(columns=['Cluster ID'])
cam.describe()

In [ ]:
scaler = preprocessing.StandardScaler()

scaler.fit(cam)
cam2 = scaler.transform(cam)

cam2_df = pd.DataFrame(cam2)
cam2_df.describe()

In [ ]:
cam2_df.plot(x=0, y=1, colormap='jet', kind='scatter')